In [1]:
import numpy as np
import math
import pandas as pd 
import matplotlib.pyplot as plt

rest1 = pd.read_csv('Rest_326.txt', sep=',', skiprows=4)
rest2 = pd.read_csv('Rest_419.txt', sep=',', skiprows=4)
rest3 = pd.read_csv('Rest_425.txt', sep=',', skiprows=4)
focus1 = pd.read_csv('Focus_326.txt', sep=',', skiprows=4)
focus2 = pd.read_csv('Focus_425.txt', sep=',', skiprows=4)
rest1 = rest1.iloc[:, 1:17]
rest2 = rest2.iloc[:, 1:17]
rest3 = rest3.iloc[:, 1:17]
rest = pd.concat([rest1, rest2, rest3], axis=0)
focus1 = focus1.iloc[:, 1:17]
focus2 = focus2.iloc[:, 1:17]
focus = pd.concat([focus1, focus2], axis=0)

datasets = [rest, focus]

def train(datasets):
    from scipy.signal import butter, filtfilt
    
    def bandpass_filter(datasets):
    
        fs = 125
        nyq = 0.5 * fs
        lowcut = 0.5 / nyq
        highcut = 50 / nyq
        filtered = []
        
        for dataset in datasets:
    
            b, a = butter(1, [lowcut, highcut], btype='band')
            filter = filtfilt(b, a, dataset)
            
            filtered.append(filter)
    
        return filtered
    
    f_datasets = bandpass_filter(datasets)
    
    def epoching(datasets):
    
        data = []
        for dataset in datasets:
            
            epoch_duration = 1
            fs = 125
            epoch_samples = epoch_duration * fs
            
            num_epochs = len(dataset) // epoch_samples
            
            epochs = []
            
            for i in range(num_epochs):
                start_index = i * epoch_samples
                end_index = start_index + epoch_samples
                epoch = dataset[start_index:end_index]
                epochs.append(epoch)
            
            # Include the remaining data points in the last epoch
            if len(dataset) % epoch_samples != 0:
                epochs.append(dataset[num_epochs * epoch_samples:])
    
            data.append(epochs)
            
        return data
        
    epoch_datasets = epoching(f_datasets)
    
    from scipy.signal import welch
    def psd_vals(epochs):
    
        psd_datasets = []
        
        for dataset in epoch_datasets:
            
            brain_waves = {'Theta': [4, 8], 'Alpha': [8, 12], 'Beta': [12, 30], 'Gamma': [30, 100]}
            psd_per_band = {}
            fs = 125
            count = 0
            for epoch in dataset:
                
                for band, (start, end) in brain_waves.items():
                    f, psd = welch(epoch.T, fs=fs, nperseg=len(epoch))
                    band_indices = np.where((f>=start) & (f<end))[0]
                    band_psd = np.mean(np.mean(psd[:, band_indices], axis=1))
                    psd_per_band[count, band] = band_psd
            
                count += 1
            
            index = pd.MultiIndex.from_tuples(psd_per_band.keys())
            df = pd.DataFrame(psd_per_band.values(), index=index, columns=['Values'])
            df = df.unstack()
            df = df['Values']
    
            psd_datasets.append(df)
            
        return psd_datasets
    
    psd_datasets = psd_vals(epoch_datasets)
    
    def fourier_coef(epoch_dataset):
    
        fc_data = []
        for epoch_data in epoch_dataset:
            
            fc = []
            for epoch in epoch_data:
            
                fourier_coefficients_channel = np.fft.fft(epoch, axis=0)
                fc.append(fourier_coefficients_channel)
            
            import pandas as pd
            
            # Assuming fourier_coefficients_epochs contains Fourier coefficients for each epoch
            
            # Initialize an empty list to store DataFrames for each epoch
            epoch_dfs = []
            
            # Loop over each epoch's Fourier coefficients
            for i, epoch_coeffs in enumerate(fc):
                # Create a DataFrame for the current epoch's Fourier coefficients
                epoch_df = pd.DataFrame(epoch_coeffs, columns=[f'Channel_{j}_Fourier_Coefficient' for j in range(epoch_coeffs.shape[1])])
                # Add an 'Epoch' column to identify the epoch
                epoch_df['Epoch'] = i
                # Append the DataFrame to the list
                epoch_dfs.append(epoch_df)
            
            # Concatenate DataFrames for all epochs into a single DataFrame
            fourier_coefficients_df = pd.concat(epoch_dfs, ignore_index=True)
            
            # Now fourier_coefficients_df contains a DataFrame with Fourier coefficients for each epoch and channel
            # Group the DataFrame by the 'Epoch' column and calculate the mean for each group
            mean_coefficients_per_epoch = fourier_coefficients_df.groupby('Epoch').mean()
    
            fc_data.append(np.real(mean_coefficients_per_epoch))
        # Now mean_coefficients_per_epoch contains the mean of each column for every epoch
        return fc_data
        
    fc_datasets = fourier_coef(epoch_datasets)
    
    from sklearn.utils import shuffle
    
    def conc(datasets):
    
        count = 0
        classes = []
        for data in datasets:
    
            y = pd.DataFrame(np.array(len(data) * [count]))
            classes.append(y)
            count += 1
        merge = pd.concat([pd.DataFrame(datasets[0]), pd.DataFrame(datasets[1])], axis=0)
        merge2 = pd.concat([classes[0], classes[1]], axis=0)
        merge3 = pd.concat([merge, merge2], axis=1)
        merge3 = merge3.rename(columns={0: 'Label'})
        
        return merge3
    
    finished_dataset_1 = conc(psd_datasets)
    finished_dataset_1 = finished_dataset_1.drop('Label', axis=1)
    finished_dataset_2 = conc(fc_datasets)
    #finished_dataset_2 = finished_dataset_2.rename(columns={0, 'Labels'})
    finished_dataset_2 = finished_dataset_2.rename(columns={'Label': 0})
    finished_dataset = pd.concat([finished_dataset_1, finished_dataset_2], axis=1)
    finished_dataset = shuffle(finished_dataset)
    X = finished_dataset.iloc[:, :20]
    X.columns = X.columns.astype(str)
    y = finished_dataset.iloc[:, 20]
    
    from sklearn.linear_model import LogisticRegression, SGDClassifier, Perceptron
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC
    from sklearn.naive_bayes import GaussianNB
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
    from xgboost import XGBClassifier
    from sklearn.model_selection import train_test_split, cross_val_score
    from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
    
    models = [LogisticRegression(), SGDClassifier(), Perceptron(), KNeighborsClassifier(), SVC(), GaussianNB(), DecisionTreeClassifier(),
     RandomForestClassifier(), AdaBoostClassifier(algorithm='SAMME'), BaggingClassifier(), XGBClassifier()]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    
        
    return X, acc, model
X, acc, model = train(datasets)
acc

0.9919571045576407

In [2]:
from scipy.signal import butter, filtfilt
def classify(data):
    
    def filtering(data):
        
        fs = 125
        nyq = 0.5 * fs
        low = 0.5 / nyq
        high = 50 / nyq
        b, a = butter(1, [low, high], btype='band')
        filter = filtfilt(b, a, data, padlen=8)
        return filter
    
    filtered_data = pd.DataFrame(filtering(data))
    
    from scipy.signal import welch
    def psd_vals(data):
    
        fs = 125
        psd_per_band = {}
        brain_waves = {'Delta': [0.5, 4], 'Theta': [4, 8], 'Alpha': [8, 12], 'Beta': [12, 30], 'Gamma': [30, 100]}
        for band, (start, end) in brain_waves.items():
            
            f, psd = welch(data.T, fs=fs, nperseg=len(data))
            indices = np.where((f >= start) & (f < end))[0]
            psd_per_band[band] = np.mean(psd[:, indices])
            
        psd_per_band = pd.DataFrame.from_dict(psd_per_band, orient='index')
        psd_per_band = psd_per_band.T

        psd_per_band = psd_per_band.sort_index(axis=1)
        psd_per_band['Delta'] = psd_per_band['Delta'].fillna(0)
        psd_per_band_na = psd_per_band
        psd_per_band = psd_per_band.drop('Delta', axis=1)
        return psd_per_band, psd_per_band_na
        
    psd_per_band, psd_per_band_na = psd_vals(filtered_data)
    psd_data = psd_per_band
    
    def fourier_coef(data):
    
        fs = 125
        
        fourier_coefficients_channel = (np.real(np.fft.fft(data, axis=0)).mean(axis=0)).reshape(1, -1)
    
        return fourier_coefficients_channel
    
    fc_data = fourier_coef(filtered_data)
    fc_data = pd.DataFrame(fc_data)
    
    dataframe = pd.concat([psd_data, fc_data], axis=1)
    dataframe.columns = dataframe.columns.astype(str)

    X = dataframe
    
    if model.predict(X) == np.array(0):

        z = 'Not Focused'

    elif model.predict(X) == np.array(1):

        z = 'Focused'

    return psd_per_band_na, z

In [3]:
import threading
import time
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import numpy as np

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Focus State and Brain Waves Dashboard"),
    dcc.Graph(id='focus-indicator'),
    dcc.Graph(id='brain-waves-bar-plot'),
    dcc.Interval(id='interval-component', interval=1000, n_intervals=0),  # Update every 1 second
    html.Button('Stop Simulation', id='stop-button', n_clicks=0)
])


@app.callback(
    [Output('focus-indicator', 'figure'),
     Output('brain-waves-bar-plot', 'figure')],
    [Input('interval-component', 'n_intervals'),
     Input('stop-button', 'n_clicks')]
)
def update_plots(data, n, stop_clicks):
    if stop_clicks > 0:
        # Stop the simulation if the stop button is clicked
        raise dash.exceptions.PreventUpdate
    
    # Call the classifier function to obtain focus state and brain waves data
    psd_per_band_na, z = classify(data)  # Pass appropriate data to the classifier function
    
    # Determine focus state based on classification result
    focus_state = 'Focused' if z == 'Focused' else 'Resting'

    # Prepare brain waves data (assuming you have calculated it in your classifier function)
    brain_waves_data = {'Frequency Band': list(psd_per_band_na.columns), 'Value': psd_per_band_na.values.flatten()}
    
    # Create a circle outline
    circle_outline = go.Scatter(x=[0], y=[0], mode='markers', marker={'size': 210, 'color': 'black'})

    # Create the focus indicator
    focus_indicator = go.Scatter(x=[0], y=[0], mode='markers',
                                 marker={'size': 200, 'color': 'green' if focus_state == 'Focused' else 'white'})

    focus_indicator_fig = {
        'data': [circle_outline, focus_indicator],
        'layout': {'title': 'Focus Indicator',
                   'xaxis': {'visible': False, 'range': [-1, 1]},
                   'yaxis': {'visible': False, 'range': [-1, 1]},
                   'showlegend': False}
    }
    
    brain_waves_fig = {
        'data': [go.Bar(x=brain_waves_data['Frequency Band'], y=brain_waves_data['Value'], 
                        marker={'color': 'blue', 'opacity': 0.8},
                        hoverinfo='none')],
        'layout': {'title': 'Brain Waves Activity',
                   'yaxis': {'range': [0, 1]},
                   'transition': {'duration': 500, 'easing': 'linear'}}
    }

    return focus_indicator_fig, brain_waves_fig


In [4]:
from brainflow.board_shim import BoardShim, BrainFlowInputParams

def receive_data(board_id, serial_port):
    params = BrainFlowInputParams()
    params.serial_port = serial_port

    BoardShim.enable_dev_board_logger()  # Enable logging if needed

    board = BoardShim(board_id, params)
    board.prepare_session()
    board.start_stream()

    first_interaction = True
    while True:
        # Retrieve data
        data = board.get_board_data()
        print(data)  # Print the acquired data
        print(len(data))

        if first_interaction = False:
        # Call update_plots function to update the dashboard
            psd_per_band_na, z = classify(data)
            update_plots(data, 0, 0)  # Pass appropriate parameters
        first_interaction = False
        # Sleep for a short interval to avoid high CPU usage
        time.sleep(1)

    # This part will never be reached because the loop runs indefinitely

    board.stop_stream()
    board.release_session()


# Start the threading for receiving data and running the dashboard
if __name__ == '__main__':
    # Replace 'CYTON_DAISY_BOARD' and '/dev/cu.usbserial-DM03H8BK' with your actual board ID and serial port
    board_id = 0  # CYTON_DAISY_BOARD
    serial_port = "/dev/cu.usbserial-DM03H8BK"

    # Create threads for receiving data and running the dashboard
    data_thread = threading.Thread(target=receive_data, args=(board_id, serial_port))
    dashboard_thread = threading.Thread(target=app.run_server, kwargs={'debug': False, 'use_reloader': False})

    # Start both threads
    data_thread.start()
    dashboard_thread.start()

    # Join threads to wait for them to finish
    data_thread.join()
    dashboard_thread.join()

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (3552238788.py, line 20)

In [5]:
from brainflow.board_shim import BoardShim, BrainFlowInputParams

def receive_data(board_id, serial_port):
    params = BrainFlowInputParams()
    params.serial_port = serial_port

    BoardShim.enable_dev_board_logger()  # Enable logging if needed

    board = BoardShim(board_id, params)
    board.prepare_session()
    board.start_stream()

    while True:
        # Retrieve data
        data = board.get_board_data()

        # Sleep for a short interval to avoid high CPU usage
        time.sleep(1)

    # This part will never be reached because the loop runs indefinitely

    board.stop_stream()
    board.release_session()


# Start the threading for receiving data and running the dashboard
if __name__ == '__main__':
    # Replace 'CYTON_DAISY_BOARD' and '/dev/cu.usbserial-DM03H8BK' with your actual board ID and serial port
    board_id = 0  # CYTON_DAISY_BOARD
    serial_port = "/dev/cu.usbserial-DM03H8BK"

    # Create threads for receiving data and running the dashboard
    data_thread = threading.Thread(target=receive_data, args=(board_id, serial_port))
    dashboard_thread = threading.Thread(target=app.run_server, kwargs={'debug': False, 'use_reloader': False})

    # Start both threads
    data_thread.start()
    dashboard_thread.start()

    # Join threads to wait for them to finish
    data_thread.join()
    dashboard_thread.join()

Exception in thread [2024-04-29 13:36:27.409] [board_logger] [info] incoming json: {
    "file": "",
    "file_anc": "",
    "file_aux": "",
    "ip_address": "",
    "ip_address_anc": "",
    "ip_address_aux": "",
    "ip_port": 0,
    "ip_port_anc": 0,
    "ip_port_aux": 0,
    "ip_protocol": 0,
    "mac_address": "",
    "master_board": -100,
    "other_info": "",
    "serial_number": "",
    "serial_port": "/dev/cu.usbserial-DM03H8BK",
    "timeout": 0
}
[2024-04-29 13:36:27.409] [board_logger] [error] Board with id 0 and the same config already exists
Thread-20 (receive_data):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/sj/v02vk_y142q73v1g1c6tn_q80000gn/T/ipykernel_2147/3111001199.py", line 10, in receive_data
  File "/opt/anaconda3/lib/python3.11/site-packa

[2024-04-29 13:36:27,799] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/dash/dash.py", line 1352, in dispatch
    ctx.run

In [2]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import numpy as np

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Focus State and Brain Waves Dashboard"),
    dcc.Graph(id='focus-indicator'),
    dcc.Graph(id='brain-waves-bar-plot'),
    dcc.Interval(id='interval-component', interval=1000, n_intervals=0),  # Update every 1 second
    html.Button('Stop Simulation', id='stop-button', n_clicks=0)
])

@app.callback(
    [Output('focus-indicator', 'figure'),
     Output('brain-waves-bar-plot', 'figure')],
    [Input('interval-component', 'n_intervals'),
     Input('stop-button', 'n_clicks')]
)
def update_plots(n, stop_clicks):
    if stop_clicks > 0:
        # Stop the simulation if the stop button is clicked
        raise dash.exceptions.PreventUpdate
    
    # Simulate focus state randomly switching between "Focused" and "Resting"
    focus_state = np.random.choice(["Focused", "Resting"])
    
    brain_waves_data = {'Frequency Band': ['Alpha', 'Beta', 'Delta', 'Theta'],
                        'Value': np.random.rand(4)}

    # Create a circle outline
    circle_outline = go.Scatter(x=[0], y=[0], mode='markers', marker={'size': 210, 'color': 'black'})

    # Create the focus indicator
    focus_indicator = go.Scatter(x=[0], y=[0], mode='markers',
                                 marker={'size': 200, 'color': 'green' if focus_state == 'Focused' else 'white'})

    focus_indicator_fig = {
        'data': [circle_outline, focus_indicator],
        'layout': {'title': 'Focus Indicator',
                   'xaxis': {'visible': False, 'range': [-1, 1]},
                   'yaxis': {'visible': False, 'range': [-1, 1]},
                   'showlegend': False}
    }
    
    brain_waves_fig = {
        'data': [go.Bar(x=brain_waves_data['Frequency Band'], y=brain_waves_data['Value'], 
                        marker={'color': 'blue', 'opacity': 0.8},
                        hoverinfo='none')],
        'layout': {'title': 'Brain Waves Activity',
                   'yaxis': {'range': [0, 1]},
                   'transition': {'duration': 500, 'easing': 'linear'}}
    }

    return focus_indicator_fig, brain_waves_fig

if __name__ == '__main__':
    app.run_server(debug=True)

In [3]:
import time
import numpy as np
from brainflow.board_shim import BoardShim, BrainFlowInputParams

def main(board_id, serial_port, classification_model):
    params = BrainFlowInputParams()
    params.serial_port = serial_port

    BoardShim.enable_dev_board_logger()  # Enable logging if needed

    board = BoardShim(board_id, params)
    board.prepare_session()
    board.start_stream()
    x = 0
    while True:
        # Retrieve data
        data = board.get_board_data()
        if x > 0 and x < 2:
        #print(data)  # Print the acquired data
            print(data)
            print(len(data[0]))
            p, z = classification_model(data)
            print(z)
        #if not data.any():
        #    continue
        x += 1
        # Sleep for a short interval to avoid high CPU usage
        time.sleep(1)

    # This part will never be reached because the loop runs indefinitely

    board.stop_stream()
    board.release_session()

    return data
# Replace 'CYTON_DAISY_BOARD' and '/dev/cu.usbserial-DM03H8BK' with your actual board ID and serial port
board_id = 0  # CYTON_DAISY_BOARD
serial_port = "/dev/cu.usbserial-DM03H8BK"

m = main(board_id, serial_port, classify)


[2024-04-30 11:00:18.778] [board_logger] [info] incoming json: {
    "file": "",
    "file_anc": "",
    "file_aux": "",
    "ip_address": "",
    "ip_address_anc": "",
    "ip_address_aux": "",
    "ip_port": 0,
    "ip_port_anc": 0,
    "ip_port_aux": 0,
    "ip_protocol": 0,
    "mac_address": "",
    "master_board": -100,
    "other_info": "",
    "serial_number": "",
    "serial_port": "/dev/cu.usbserial-DM03H8BK",
    "timeout": 0
}
[2024-04-30 11:00:18.778] [board_logger] [trace] Board object created 0
[2024-04-30 11:00:18.778] [board_logger] [info] opening port /dev/cu.usbserial-DM03H8BK
[2024-04-30 11:00:18.798] [board_logger] [trace] port /dev/cu.usbserial-DM03H8BK is open
[2024-04-30 11:00:18.813] [board_logger] [trace] set port settings
[2024-04-30 11:00:18.815] [board_logger] [info] set_latency_res is: 0
[2024-04-30 11:00:18.815] [board_logger] [debug] sending v to the board
[2024-04-30 11:00:20.003] [board_logger] [debug] sending d to the board
[2024-04-30 11:00:21.102] [

[[ 0.00000000e+00  1.00000000e+00  2.00000000e+00 ...  2.23000000e+02
   2.24000000e+02  2.25000000e+02]
 [-9.37500112e+04 -1.87500022e+05 -1.87500022e+05 ... -1.87500022e+05
  -1.87500022e+05 -1.87500022e+05]
 [-9.37500112e+04 -1.87500022e+05 -1.87500022e+05 ... -1.87500022e+05
  -1.87500022e+05 -1.87500022e+05]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.71450002e+09  1.71450002e+09  1.71450002e+09 ...  1.71450002e+09
   1.71450002e+09  1.71450002e+09]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
226


/opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- 100
- 101
- 102
- 103
- 104
- ...


In [3]:
import dash
from dash import dcc, html, Input, Output
import dash.exceptions
import plotly.graph_objs as go
import numpy as np
import pandas as pd
import time
from brainflow.board_shim import BoardShim, BrainFlowInputParams

def main(board_id, serial_port, classification_algo, model):
    params = BrainFlowInputParams()
    params.serial_port = serial_port

    BoardShim.enable_dev_board_logger()  # Enable logging if needed

    board = BoardShim(board_id, params)
    board.prepare_session()
    board.start_stream()
    num = 0
    while True:
        # Retrieve data
        data = board.get_board_data()
        data = pd.DataFrame(data)
        data = data.T
        data = data.iloc[:, 1:9]
        if x > 0 and x < 3:
            p, z = classification_algo(data, model)
            focus_state = "Focused" if z == 'Focused' else "Resting"
            focus_indicator = go.Scatter(x=[0], y=[0], mode='markers',
                                         marker={'size': 200, 'color': 'green' if focus_state == 'Focused' else 'white'})
            focus_indicator_fig = {
                'data': [go.Scatter(x=[0], y=[0], mode='markers', marker={'size': 210, 'color': 'black'}), focus_indicator],
                'layout': {'title': 'Focus Indicator',
                           'xaxis': {'visible': False, 'range': [-1, 1]},
                           'yaxis': {'visible': False, 'range': [-1, 1]},
                           'showlegend': False}
            }
            plt.show()  # Only update the focus indicator

        num += 1
    # Sleep for a short interval to avoid high CPU usage
        time.sleep(1)
    # This part will never be reached because the loop runs indefinitely

    board.stop_stream()
    board.release_session()

# Replace 'CYTON_DAISY_BOARD' and '/dev/cu.usbserial-DM03H8BK' with your actual board ID and serial port
board_id = 0  # CYTON_DAISY_BOARD
serial_port = "/dev/cu.usbserial-DM03H8BK"

main(board_id, serial_port, classify, model)

[2024-04-30 12:39:02.374] [board_logger] [info] incoming json: {
    "file": "",
    "file_anc": "",
    "file_aux": "",
    "ip_address": "",
    "ip_address_anc": "",
    "ip_address_aux": "",
    "ip_port": 0,
    "ip_port_anc": 0,
    "ip_port_aux": 0,
    "ip_protocol": 0,
    "mac_address": "",
    "master_board": -100,
    "other_info": "",
    "serial_number": "",
    "serial_port": "/dev/cu.usbserial-DM03H8BK",
    "timeout": 0
}
[2024-04-30 12:39:02.374] [board_logger] [trace] Board object created 0
[2024-04-30 12:39:02.374] [board_logger] [info] opening port /dev/cu.usbserial-DM03H8BK
[2024-04-30 12:39:02.394] [board_logger] [trace] port /dev/cu.usbserial-DM03H8BK is open
[2024-04-30 12:39:02.409] [board_logger] [trace] set port settings
[2024-04-30 12:39:02.411] [board_logger] [info] set_latency_res is: 0
[2024-04-30 12:39:02.411] [board_logger] [debug] sending v to the board
[2024-04-30 12:39:03.576] [board_logger] [debug] sending d to the board
[2024-04-30 12:39:04.677] [

({'data': [Scatter({
       'marker': {'color': 'black', 'size': 210}, 'mode': 'markers', 'x': [0], 'y': [0]
   }),
   Scatter({
       'marker': {'color': 'green', 'size': 200}, 'mode': 'markers', 'x': [0], 'y': [0]
   })],
  'layout': {'title': 'Focus Indicator',
   'xaxis': {'visible': False, 'range': [-1, 1]},
   'yaxis': {'visible': False, 'range': [-1, 1]},
   'showlegend': False}},
 None)

In [11]:
import pandas as pd
import pandas as pd

# Assuming your DataFrame is named df
pd.set_option('display.max_columns', None)
#print(df.head())

new_data = pd.read_csv('data_raw_2.csv').T
new_data = new_data.iloc[:, 0:8]
new_data

,0,1,2,3,4,5,6,7
0.0,-9277.225647,-27440.655522,-32684.978269,-93750.011176,-12175.598702,-4618.943288,-17958.643193,-25677.862844
1.0,20872.148379,3083.780776,-187500.022352,-187500.022352,-21205.547000,-37624.803200,-44341.524761,-36091.071199
2.0,-20102.711928,-53914.352228,-65271.988842,-187500.022352,-23359.718723,-8226.984230,-34781.571899,-50141.310709
3.0,19725.950924,2156.183381,-187500.022352,-187500.022352,-19188.883208,-37007.135094,-44915.115227,-35560.172565
4.0,-19010.985674,-55024.183395,-65889.254616,-187500.022352,-24289.551293,-9098.478746,-35252.254933,-50773.708615
...,...,...,...,...,...,...,...,...
222.0,-19680.867455,-55311.582126,-64991.273283,-187500.022352,-23962.075885,-8914.658000,-37241.806059,-50947.158152
223.0,20690.182828,3109.529985,-187500.022352,-187500.022352,-20117.240562,-36944.885486,-44376.639351,-35700.697982
224.0,-20826.886097,-55174.789450,-65651.633221,-187500.022352,-23586.700688,-8414.850642,-35931.949131,-49780.486498
225.0,20123.990789,2414.457788,-187500.022352,-187500.022352,-20084.316443,-37533.652786,-44428.584806,-35989.281355


In [1]:
import numpy as np
import math
import pandas as pd 
import matplotlib.pyplot as plt

rest1 = pd.read_csv('Rest_326.txt', sep=',', skiprows=4)
rest2 = pd.read_csv('Rest_419.txt', sep=',', skiprows=4)
rest3 = pd.read_csv('Rest_425.txt', sep=',', skiprows=4)
focus1 = pd.read_csv('Focus_326.txt', sep=',', skiprows=4)
focus2 = pd.read_csv('Focus_425.txt', sep=',', skiprows=4)
rest1_original = rest1
rest1 = rest1.iloc[:, 1:9]
rest2 = rest2.iloc[:, 1:9]
rest3 = rest3.iloc[:, 1:9]
rest = pd.concat([rest1, rest2, rest3], axis=0)
focus1 = focus1.iloc[:, 1:9]
focus2 = focus2.iloc[:, 1:9]
focus = pd.concat([focus1, focus2], axis=0)

datasets = [rest, focus]

from scipy.signal import butter, filtfilt

def bandpass_filter(datasets):

    fs = 125
    nyq = 0.5 * fs
    lowcut = 0.5 / nyq
    highcut = 50 / nyq
    filtered = []
    
    for dataset in datasets:
        print(len(dataset))
        b, a = butter(1, [lowcut, highcut], btype='band')
        filter = filtfilt(b, a, dataset, padlen=7)
        
        filtered.append(filter)

    return filtered

f_datasets = bandpass_filter(datasets)

def epoching(datasets):

    data = []
    for dataset in datasets:
        
        epoch_duration = 1
        fs = 125
        epoch_samples = epoch_duration * fs
        epoch_samples = 227
        num_epochs = len(dataset) // epoch_samples
        
        epochs = []
        
        for i in range(num_epochs):
            start_index = i * epoch_samples
            end_index = start_index + epoch_samples
            epoch = dataset[start_index:end_index]
            epochs.append(epoch)
        
        # Include the remaining data points in the last epoch
        #if len(dataset) % epoch_samples != 0:
         #   epochs.append(dataset[num_epochs * epoch_samples:])

        data.append(epochs)
        
    return data
    
epoch_datasets = epoching(f_datasets)

from scipy.signal import welch
def psd_vals(epochs):

    psd_datasets = []
    
    for dataset in epoch_datasets:
        
        brain_waves = {'Theta': [4, 8], 'Alpha': [8, 12], 'Beta': [12, 30], 'Gamma': [30, 100]}
        psd_per_band = {}
        fs = 125
        count = 0
        for epoch in dataset:
            
            for band, (start, end) in brain_waves.items():
                f, psd = welch(epoch.T, fs=fs, nperseg=len(epoch))
                band_indices = np.where((f>=start) & (f<end))[0]
                band_psd = np.mean(np.mean(psd[:, band_indices], axis=1))
                psd_per_band[count, band] = band_psd
        
            count += 1
        
        index = pd.MultiIndex.from_tuples(psd_per_band.keys())
        df = pd.DataFrame(psd_per_band.values(), index=index, columns=['Values'])
        df = df.unstack()
        df = df['Values']

        psd_datasets.append(df)
        
    return psd_datasets

psd_datasets = psd_vals(epoch_datasets)
psd_datasets[0]

def fourier_coef(epoch_dataset):

    fc_data = []
    for epoch_data in epoch_dataset:
        
        fc = []
        for epoch in epoch_data:
        
            fourier_coefficients_channel = np.fft.fft(epoch, axis=0)
            fc.append(fourier_coefficients_channel)
        
        import pandas as pd
        
        # Assuming fourier_coefficients_epochs contains Fourier coefficients for each epoch
        
        # Initialize an empty list to store DataFrames for each epoch
        epoch_dfs = []
        
        # Loop over each epoch's Fourier coefficients
        for i, epoch_coeffs in enumerate(fc):
            # Create a DataFrame for the current epoch's Fourier coefficients
            epoch_df = pd.DataFrame(epoch_coeffs, columns=[f'Channel_{j}_Fourier_Coefficient' for j in range(epoch_coeffs.shape[1])])
            # Add an 'Epoch' column to identify the epoch
            epoch_df['Epoch'] = i
            # Append the DataFrame to the list
            epoch_dfs.append(epoch_df)
        
        # Concatenate DataFrames for all epochs into a single DataFrame
        fourier_coefficients_df = pd.concat(epoch_dfs, ignore_index=True)
        
        # Now fourier_coefficients_df contains a DataFrame with Fourier coefficients for each epoch and channel
        # Group the DataFrame by the 'Epoch' column and calculate the mean for each group
        mean_coefficients_per_epoch = fourier_coefficients_df.groupby('Epoch').mean()

        fc_data.append(np.real(mean_coefficients_per_epoch))
    # Now mean_coefficients_per_epoch contains the mean of each column for every epoch
    return fc_data
    
fc_datasets = fourier_coef(epoch_datasets)
pd.DataFrame(fc_datasets[0])

from sklearn.utils import shuffle

def conc(datasets):

    count = 0
    classes = []
    for data in datasets:

        y = pd.DataFrame(np.array(len(data) * [count]))
        classes.append(y)
        count += 1
    merge = pd.concat([pd.DataFrame(datasets[0]), pd.DataFrame(datasets[1])], axis=0)
    merge2 = pd.concat([classes[0], classes[1]], axis=0)
    merge3 = pd.concat([merge, merge2], axis=1)
    merge3 = merge3.rename(columns={0: 'Label'})
    #merge3 = shuffle(merge3)
    
    return merge3

finished_dataset_1 = conc(psd_datasets)
finished_dataset_1 = finished_dataset_1.drop('Label', axis=1)
finished_dataset_2 = conc(fc_datasets)
#finished_dataset_2 = finished_dataset_2.rename(columns={0, 'Labels'})
finished_dataset_2 = finished_dataset_2.rename(columns={'Label': 0})
finished_dataset = pd.concat([finished_dataset_1, finished_dataset_2], axis=1)
finished_dataset = shuffle(finished_dataset)
X = finished_dataset.iloc[:, :12]
X.columns = X.columns.astype(str)
y = finished_dataset.iloc[:, 12]
from sklearn.linear_model import LogisticRegression, SGDClassifier, Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

model = RandomForestClassifier()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_train = model.predict(X_train)
test_acc = accuracy_score(y_test, y_pred)
train_acc = accuracy_score(y_train, y_pred_train)
test_class = classification_report(y_test, y_pred)
train_class = classification_report(y_train, y_pred_train)

114796
118206


In [2]:
from scipy.signal import butter, filtfilt
def classify(data, model):
    
    def filtering(data):
        
        fs = 125
        nyq = 0.5 * fs
        low = 0.5 / nyq
        high = 50 / nyq
        b, a = butter(1, [low, high], btype='band')
        filter = filtfilt(b, a, data, padlen=7)
        return filter
    
    filtered_data = pd.DataFrame(filtering(data))
    
    from scipy.signal import welch
    def psd_vals(data):
    
        fs = 125
        psd_per_band = {}
        brain_waves = {'Delta': [0.5, 4], 'Theta': [4, 8], 'Alpha': [8, 12], 'Beta': [12, 30], 'Gamma': [30, 100]}
        for band, (start, end) in brain_waves.items():
            
            f, psd = welch(data.T, fs=fs, nperseg=len(data))
            indices = np.where((f >= start) & (f < end))[0]
            psd_per_band[band] = np.mean(psd[:, indices])
            
        psd_per_band = pd.DataFrame.from_dict(psd_per_band, orient='index')
        psd_per_band = psd_per_band.T

        psd_per_band = psd_per_band.sort_index(axis=1)
        psd_per_band['Delta'] = psd_per_band['Delta'].fillna(0)
        psd_per_band_na = psd_per_band
        psd_per_band = psd_per_band.drop('Delta', axis=1)
        return psd_per_band, psd_per_band_na
        
    psd_per_band, psd_per_band_na = psd_vals(filtered_data)
    psd_data = psd_per_band
    
    def fourier_coef(data):
    
        fs = 125
        
        fourier_coefficients_channel = (np.real(np.fft.fft(data, axis=0)).mean(axis=0)).reshape(1, -1)
    
        return fourier_coefficients_channel
    
    fc_data = fourier_coef(filtered_data)
    fc_data = pd.DataFrame(fc_data)
    
    dataframe = pd.concat([psd_data, fc_data], axis=1)
    dataframe.columns = dataframe.columns.astype(str)

    X = dataframe
    
    if model.predict(X) == np.array(0):

        z = 'Not Focused'

    elif model.predict(X) == np.array(1):

        z = 'Focused'

    return psd_per_band_na, z

classify(new_data, model)

NameError: name 'new_data' is not defined

In [1]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import numpy as np
import pandas as pd
import time
from brainflow.board_shim import BoardShim, BrainFlowInputParams
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from scipy.signal import butter, filtfilt
from scipy.signal import welch

app = dash.Dash(__name__)

import numpy as np
import math
import pandas as pd 
import matplotlib.pyplot as plt

rest1 = pd.read_csv('Rest_326.txt', sep=',', skiprows=4)
rest2 = pd.read_csv('Rest_419.txt', sep=',', skiprows=4)
rest3 = pd.read_csv('Rest_425.txt', sep=',', skiprows=4)
focus1 = pd.read_csv('Focus_326.txt', sep=',', skiprows=4)
focus2 = pd.read_csv('Focus_425.txt', sep=',', skiprows=4)
rest1_original = rest1
rest1 = rest1.iloc[:, 1:9]
rest2 = rest2.iloc[:, 1:9]
rest3 = rest3.iloc[:, 1:9]
rest = pd.concat([rest1, rest2, rest3], axis=0)
focus1 = focus1.iloc[:, 1:9]
focus2 = focus2.iloc[:, 1:9]
focus = pd.concat([focus1, focus2], axis=0)

datasets = [rest, focus]

from scipy.signal import butter, filtfilt

def bandpass_filter(datasets):

    fs = 125
    nyq = 0.5 * fs
    lowcut = 0.5 / nyq
    highcut = 50 / nyq
    filtered = []
    
    for dataset in datasets:
        print(len(dataset))
        b, a = butter(1, [lowcut, highcut], btype='band')
        filter = filtfilt(b, a, dataset, padlen=7)
        
        filtered.append(filter)

    return filtered

f_datasets = bandpass_filter(datasets)

def epoching(datasets):

    data = []
    for dataset in datasets:
        
        epoch_duration = 1
        fs = 125
        epoch_samples = epoch_duration * fs
        epoch_samples = 227
        num_epochs = len(dataset) // epoch_samples
        
        epochs = []
        
        for i in range(num_epochs):
            start_index = i * epoch_samples
            end_index = start_index + epoch_samples
            epoch = dataset[start_index:end_index]
            epochs.append(epoch)
        
        # Include the remaining data points in the last epoch
        #if len(dataset) % epoch_samples != 0:
         #   epochs.append(dataset[num_epochs * epoch_samples:])

        data.append(epochs)
        
    return data
    
epoch_datasets = epoching(f_datasets)

from scipy.signal import welch
def psd_vals(epochs):

    psd_datasets = []
    
    for dataset in epoch_datasets:
        
        brain_waves = {'Theta': [4, 8], 'Alpha': [8, 12], 'Beta': [12, 30], 'Gamma': [30, 100]}
        psd_per_band = {}
        fs = 125
        count = 0
        for epoch in dataset:
            
            for band, (start, end) in brain_waves.items():
                f, psd = welch(epoch.T, fs=fs, nperseg=len(epoch))
                band_indices = np.where((f>=start) & (f<end))[0]
                band_psd = np.mean(np.mean(psd[:, band_indices], axis=1))
                psd_per_band[count, band] = band_psd
        
            count += 1
        
        index = pd.MultiIndex.from_tuples(psd_per_band.keys())
        df = pd.DataFrame(psd_per_band.values(), index=index, columns=['Values'])
        df = df.unstack()
        df = df['Values']

        psd_datasets.append(df)
        
    return psd_datasets

psd_datasets = psd_vals(epoch_datasets)
psd_datasets[0]

def fourier_coef(epoch_dataset):

    fc_data = []
    for epoch_data in epoch_dataset:
        
        fc = []
        for epoch in epoch_data:
        
            fourier_coefficients_channel = np.fft.fft(epoch, axis=0)
            fc.append(fourier_coefficients_channel)
        
        import pandas as pd
        
        # Assuming fourier_coefficients_epochs contains Fourier coefficients for each epoch
        
        # Initialize an empty list to store DataFrames for each epoch
        epoch_dfs = []
        
        # Loop over each epoch's Fourier coefficients
        for i, epoch_coeffs in enumerate(fc):
            # Create a DataFrame for the current epoch's Fourier coefficients
            epoch_df = pd.DataFrame(epoch_coeffs, columns=[f'Channel_{j}_Fourier_Coefficient' for j in range(epoch_coeffs.shape[1])])
            # Add an 'Epoch' column to identify the epoch
            epoch_df['Epoch'] = i
            # Append the DataFrame to the list
            epoch_dfs.append(epoch_df)
        
        # Concatenate DataFrames for all epochs into a single DataFrame
        fourier_coefficients_df = pd.concat(epoch_dfs, ignore_index=True)
        
        # Now fourier_coefficients_df contains a DataFrame with Fourier coefficients for each epoch and channel
        # Group the DataFrame by the 'Epoch' column and calculate the mean for each group
        mean_coefficients_per_epoch = fourier_coefficients_df.groupby('Epoch').mean()

        fc_data.append(np.real(mean_coefficients_per_epoch))
    # Now mean_coefficients_per_epoch contains the mean of each column for every epoch
    return fc_data
    
fc_datasets = fourier_coef(epoch_datasets)
pd.DataFrame(fc_datasets[0])

from sklearn.utils import shuffle

def conc(datasets):

    count = 0
    classes = []
    for data in datasets:

        y = pd.DataFrame(np.array(len(data) * [count]))
        classes.append(y)
        count += 1
    merge = pd.concat([pd.DataFrame(datasets[0]), pd.DataFrame(datasets[1])], axis=0)
    merge2 = pd.concat([classes[0], classes[1]], axis=0)
    merge3 = pd.concat([merge, merge2], axis=1)
    merge3 = merge3.rename(columns={0: 'Label'})
    #merge3 = shuffle(merge3)
    
    return merge3

finished_dataset_1 = conc(psd_datasets)
finished_dataset_1 = finished_dataset_1.drop('Label', axis=1)
finished_dataset_2 = conc(fc_datasets)
#finished_dataset_2 = finished_dataset_2.rename(columns={0, 'Labels'})
finished_dataset_2 = finished_dataset_2.rename(columns={'Label': 0})
finished_dataset = pd.concat([finished_dataset_1, finished_dataset_2], axis=1)
finished_dataset = shuffle(finished_dataset)
X = finished_dataset.iloc[:, :12]
X.columns = X.columns.astype(str)
y = finished_dataset.iloc[:, 12]
from sklearn.linear_model import LogisticRegression, SGDClassifier, Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

model = RandomForestClassifier()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_train = model.predict(X_train)
test_acc = accuracy_score(y_test, y_pred)
train_acc = accuracy_score(y_train, y_pred_train)
test_class = classification_report(y_test, y_pred)
train_class = classification_report(y_train, y_pred_train)

app.layout = html.Div([
    html.Button('Stop', id='stop-button', n_clicks=0),
    dcc.Graph(id='focus-indicator', style={'height': '400px', 'width': '30%', 'display': 'inline-block'}),
    dcc.Graph(id='brainwave-bar', style={'height': '400px', 'width': '50%', 'display': 'inline-block'}),
    dcc.Interval(
        id='interval-component',
        interval=1*1000,  # in milliseconds
        n_intervals=0
    )
])

# Create board object outside the callback function
params = BrainFlowInputParams()
params.serial_port = "/dev/cu.usbserial-DM03H8BK"
board = BoardShim(0, params)  # assuming board_id = 0
board.prepare_session()
board.start_stream()

from scipy.signal import butter, filtfilt
def classify(data, model):
    
    def filtering(data):
        
        fs = 125
        nyq = 0.5 * fs
        low = 0.5 / nyq
        high = 50 / nyq
        b, a = butter(1, [low, high], btype='band')
        filter = filtfilt(b, a, data, padlen=7)
        return filter
    
    filtered_data = pd.DataFrame(filtering(data))
    
    from scipy.signal import welch
    def psd_vals(data):
    
        fs = 125
        psd_per_band = {}
        brain_waves = {'Delta': [0.5, 4], 'Theta': [4, 8], 'Alpha': [8, 12], 'Beta': [12, 30], 'Gamma': [30, 100]}
        for band, (start, end) in brain_waves.items():
            
            f, psd = welch(data.T, fs=fs, nperseg=len(data))
            indices = np.where((f >= start) & (f < end))[0]
            psd_per_band[band] = np.mean(psd[:, indices])
            
        psd_per_band = pd.DataFrame.from_dict(psd_per_band, orient='index')
        psd_per_band = psd_per_band.T

        psd_per_band = psd_per_band.sort_index(axis=1)
        psd_per_band['Delta'] = psd_per_band['Delta'].fillna(0)
        psd_per_band_na = psd_per_band
        psd_per_band = psd_per_band.drop('Delta', axis=1)
        return psd_per_band, psd_per_band_na
        
    psd_per_band, psd_per_band_na = psd_vals(filtered_data)
    psd_data = psd_per_band
    
    def fourier_coef(data):
    
        fs = 125
        
        fourier_coefficients_channel = (np.real(np.fft.fft(data, axis=0)).mean(axis=0)).reshape(1, -1)
    
        return fourier_coefficients_channel
    
    fc_data = fourier_coef(filtered_data)
    fc_data = pd.DataFrame(fc_data)
    
    dataframe = pd.concat([psd_data, fc_data], axis=1)
    dataframe.columns = dataframe.columns.astype(str)

    X = dataframe
    
    if model.predict(X) == np.array(0):

        z = 'Not Focused'

    elif model.predict(X) == np.array(1):

        z = 'Focused'

    return psd_per_band_na, z
    
@app.callback(
    [Output('focus-indicator', 'figure'),
    Output('brainwave-bar', 'figure')],
    [Input('interval-component', 'n_intervals'),
     Input('stop-button', 'n_clicks')]
)
def update_focus_indicator(n, stop_clicks):
    if stop_clicks > 0:
        board.stop_stream()
        board.release_session()
        raise dash.exceptions.PreventUpdate

    # Retrieve data
    data = board.get_board_data()
    data = pd.DataFrame(data)
    data = data.T
    data = data.iloc[:, 1:9]
    print(data)
    p, z = classify(data, model)
    focus_state = "Focused" if z == 'Focused' else "Resting"
    focus_indicator = go.Scatter(x=[0], y=[0], mode='markers',
                                 marker={'size': 200, 'color': 'green' if focus_state == 'Focused' else 'white'})
    focus_indicator_fig = {
        'data': [go.Scatter(x=[0], y=[0], mode='markers', marker={'size': 210, 'color': 'black'}), focus_indicator],
        'layout': {'title': 'Focus Indicator',
                   'xaxis': {'visible': False, 'range': [-1, 1]},
                   'yaxis': {'visible': False, 'range': [-1, 1]},
                   'showlegend': False}
    }

    # Create bar plot for brainwave data
    brainwave_bar = go.Figure()
    brainwave_bar.add_trace(go.Bar(
        x=['Delta', 'Theta', 'Alpha'],
        y=p.values.flatten(),
        marker_color=['red', 'blue', 'green']
    ))
    brainwave_bar.update_layout(title='Brainwave Activity', xaxis_title='Frequency Band', yaxis_title='Power')

    return focus_indicator_fig, brainwave_bar

if __name__ == '__main__':
    #app.run_server(debug=True)


SyntaxError: incomplete input (3120479899.py, line 335)